In [ ]:
# Student Name: Kanashima Hatsumi
# Student ID: 1004429

## Load the IMDB dataset and create the vocabulary

In [1]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

EMBEDDING_DIM=50
VOCAB_SIZE=20000

# Get IMDB dataset
imdb = IMDB(split='train')

# Load English tokenizer, tagger, parser and NER
tokenizer = get_tokenizer('spacy', language='en') # Split the text into words

# build the vocab
counter = Counter()
for i, (label, line) in enumerate(imdb):
    counter.update(tokenizer(line))

ordered_dict = OrderedDict(counter.most_common()[:VOCAB_SIZE])
vocab = vocab(ordered_dict)

# insert special tokens and set default index to 'unknown'
vocab.insert_token('<PAD>', 0) # Padding
vocab.insert_token('<UNK>', 1) # For unknown word
vocab.set_default_index(1)

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:06<00:00, 13.1MB/s]


## Create embedding vectors from GloVe

In [2]:
import torchtext as text

# load glove embeddings
vec = text.vocab.GloVe(name='6B', dim=50)
# create the embedding matrix, a torch tensor in the shape (num_words+1, embedding_dim)
word_emb = vec.get_vecs_by_tokens(vocab.get_itos())

.vector_cache/glove.6B.zip: 862MB [02:42, 5.29MB/s]                           
100%|█████████▉| 399999/400000 [00:14<00:00, 27690.75it/s]


## Build up train/test dataset

In [3]:
import torch
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data.dataset import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# transform input text and label to ids
def process_text(text):
    return vocab(tokenizer(text))

label_to_ids = {'pos':0, 'neg':1}

# preprocess a batch of raw data (string-like text/labels) into tensors
def collate_batch(batch): # Creates Batches
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_to_ids[_label]) # Converting strings into 1s and 0s
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64) # Convert text into sequences
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    # label must be in the same size as target
    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

# train_iter, test_iter = IMDB()
train_iter, test_iter = IMDB()

# transform datasets iterator into map style so that they can be repeatedly loaded in a new epoch
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.7)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                             shuffle=True, collate_fn=collate_batch)
# Question 1
# https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
val_dataloader = DataLoader(split_valid_, batch_size=128,
                             shuffle=True, collate_fn=collate_batch) # Write a validation dataloader by spliting the training data

## Define the logistic regression model

In [33]:
# logistic model
import torch
import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        
        # Question 2 : Replace the EmbeddingBag using PyTorch builtin functions that does the same job of computing sentence representation by taking average of the word embeddings.

        # self.embedding = nn.EmbeddingBag(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
        self.embedding = nn.Embedding(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
     
        # Question 3 : Write a Fully Connected Layer (FC layer) with output size of 100 followed by a non-linear activation e.g., ReLU
        self.fc1 = nn.Linear(embed_dim, 100)
        self.fc2 = nn.Linear(100, 1)
        
        self.fcLayers = nn.Sequential(
                        self.fc1,
                        nn.ReLU(),
                        self.fc2
                        )
        self._init_weights()

    def _init_weights(self):
        """Initialize network parameters 
        """
        initrange = 0.5
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.embedding.weight.data.uniform_(-initrange, initrange)

    def forward(self, text, offsets):
        embedded = self.embedding(text)

        ls = []
        for offset in range(len(offsets)):
          if offsets[offset] == offsets[-1]:
              mean = torch.mean(embedded[offsets[offset]:], dim=0)
          else:
              mean = torch.mean(embedded[offsets[offset]: offsets[offset+1]], dim=0)

          ls.append(mean)    
        embedded = torch.stack(ls, dim=0)
          
        # Question 4: Use the new model you define in __init__()
        return torch.sigmoid(self.fcLayers(embedded))

## Define train and test function

In [25]:
import time

def train(train_dataloader, val_dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text, offsets)
        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update parameters by stepping the optimizer
        optimizer.step()
        total_acc += ((predicted_label > 0.5) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
          elapsed = time.time() - start_time
          print('| epoch {:3d} | {:5d}/{:5d} batches '
                '| accuracy {:8.3f}'.format(epoch, idx, len(train_dataloader),
                                            total_acc/total_count))
          total_acc, total_count = 0, 0
          start_time = time.time()

    # Early stopping
    current_acc = validation(val_dataloader)
    print("Validation Accuracy: ", current_acc)

    return current_acc

def validation(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count



In [26]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Train and evaluate the model for several epochs

In [34]:
# Hyperparameters
EPOCHS = 200 # epoch
patience = 10

model = LogisticRegression(word_vec=word_emb, embed_dim=EMBEDDING_DIM).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()
total_accu = None
max_val_acc = 0
stop_trigger = 0

for epoch in range(1, EPOCHS + 1):
    print("Epoch:" , epoch)
    epoch_start_time = time.time()
    current_val_acc = train(train_dataloader, val_dataloader)
    
    # Question 5: Use your validation set to early stop the model. Remember to early stop when the validation accuracy does not improve for continous N number of epochs where N is a hyperparameter. Set N = 10
    if (current_val_acc >= max_val_acc):
      max_val_acc = current_val_acc
      stop_trigger = 0 # Reset count
    
    elif (current_val_acc < max_val_acc):
      stop_trigger += 1 
      print("Triggered! --> ", stop_trigger , "/", patience)

    if stop_trigger == patience:
        print("Early stopped!!!")
        break
    print("Max Validation Accuracy: ", max_val_acc)
    
    

Epoch: 1
| epoch   1 |    50/  137 batches | accuracy    0.778
| epoch   1 |   100/  137 batches | accuracy    0.876
Validation Accuracy:  63.6512
Max Validation Accuracy:  63.6512
Epoch: 2
| epoch   2 |    50/  137 batches | accuracy    0.948
| epoch   2 |   100/  137 batches | accuracy    0.939
Validation Accuracy:  63.63733333333333
Triggered! -->  1 / 10
Max Validation Accuracy:  63.6512
Epoch: 3
| epoch   3 |    50/  137 batches | accuracy    0.976
| epoch   3 |   100/  137 batches | accuracy    0.974
Validation Accuracy:  63.6096
Triggered! -->  2 / 10
Max Validation Accuracy:  63.6512
Epoch: 4
| epoch   4 |    50/  137 batches | accuracy    0.988
| epoch   4 |   100/  137 batches | accuracy    0.987
Validation Accuracy:  63.6512
Max Validation Accuracy:  63.6512
Epoch: 5
| epoch   5 |    50/  137 batches | accuracy    0.995
| epoch   5 |   100/  137 batches | accuracy    0.987
Validation Accuracy:  63.699733333333334
Max Validation Accuracy:  63.699733333333334
Epoch: 6
| epoch 

In [35]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.2f}%'.format(accu_test))

test accuracy    63.94%
